In [1]:
!pip install simpletransformers

In [2]:
import pandas as pd
data = pd.read_csv("/content/custom_ent_bert_data.csv",encoding="latin1" )

In [3]:
data.head(30)

,Sentence #,text,word,tag
0,Sentence 1,Heat stress and persistent dehydration can cau...,Heat,O
1,Sentence 1,Heat stress and persistent dehydration can cau...,stress,base
2,Sentence 1,Heat stress and persistent dehydration can cau...,persistent,O
3,Sentence 1,Heat stress and persistent dehydration can cau...,dehydration,base
4,Sentence 1,Heat stress and persistent dehydration can cau...,can,O
5,Sentence 1,Heat stress and persistent dehydration can cau...,cause,O
6,Sentence 1,Heat stress and persistent dehydration can cau...,kidney,base
7,Sentence 1,Heat stress and persistent dehydration can cau...,damage,O
8,Sentence 2,"This summer, the heat was intensified by a lac...",This summer,O
9,Sentence 2,"This summer, the heat was intensified by a lac...",heat,base


In [4]:
data = data.fillna(method ="ffill")

In [5]:
data.head(30)

,Sentence #,text,word,tag
0,Sentence 1,Heat stress and persistent dehydration can cau...,Heat,O
1,Sentence 1,Heat stress and persistent dehydration can cau...,stress,base
2,Sentence 1,Heat stress and persistent dehydration can cau...,persistent,O
3,Sentence 1,Heat stress and persistent dehydration can cau...,dehydration,base
4,Sentence 1,Heat stress and persistent dehydration can cau...,can,O
5,Sentence 1,Heat stress and persistent dehydration can cau...,cause,O
6,Sentence 1,Heat stress and persistent dehydration can cau...,kidney,base
7,Sentence 1,Heat stress and persistent dehydration can cau...,damage,O
8,Sentence 2,"This summer, the heat was intensified by a lac...",This summer,O
9,Sentence 2,"This summer, the heat was intensified by a lac...",heat,base


In [6]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [7]:
data["Sentence #"] = LabelEncoder().fit_transform(data["Sentence #"] )

In [8]:
data.head(30)

,Sentence #,text,word,tag
0,0,Heat stress and persistent dehydration can cau...,Heat,O
1,0,Heat stress and persistent dehydration can cau...,stress,base
2,0,Heat stress and persistent dehydration can cau...,persistent,O
3,0,Heat stress and persistent dehydration can cau...,dehydration,base
4,0,Heat stress and persistent dehydration can cau...,can,O
5,0,Heat stress and persistent dehydration can cau...,cause,O
6,0,Heat stress and persistent dehydration can cau...,kidney,base
7,0,Heat stress and persistent dehydration can cau...,damage,O
8,11,"This summer, the heat was intensified by a lac...",This summer,O
9,11,"This summer, the heat was intensified by a lac...",heat,base


In [9]:
data.rename(columns={"Sentence #":"sentence_id", "word":"words", "tag":"labels"}, inplace =True)

In [10]:
data

,sentence_id,text,words,labels
0,0,Heat stress and persistent dehydration can cau...,Heat,O
1,0,Heat stress and persistent dehydration can cau...,stress,base
2,0,Heat stress and persistent dehydration can cau...,persistent,O
3,0,Heat stress and persistent dehydration can cau...,dehydration,base
4,0,Heat stress and persistent dehydration can cau...,can,O
...,...,...,...,...
690,56,"As heatwaves have increased, kelp forests, sea...",meadows,base
691,56,"As heatwaves have increased, kelp forests, sea...",coral,O
692,56,"As heatwaves have increased, kelp forests, sea...",reefs,base
693,56,"As heatwaves have increased, kelp forests, sea...",have,O


In [11]:
data["labels"] = data["labels"].str.upper()

In [12]:
X = data[["sentence_id", "words"]]
Y = data["labels"]

In [13]:
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size =0.2)

In [14]:
#building up train data and test data
train_data = pd.DataFrame({"sentence_id":x_train["sentence_id"],"words":x_train["words"],"labels":y_train})
test_data = pd.DataFrame({"sentence_id":x_test["sentence_id"],"words":x_test["words"],"labels":y_test})

In [15]:
train_data

,sentence_id,words,labels
430,35,neurological,O
311,23,hotter,O
604,48,level,O
380,29,strokes,BASE
100,59,$8 trillion,O
...,...,...,...
685,56,have,O
348,26,increasing,O
39,44,pollution,BASE
487,39,groundwater,BASE


# Model Training


In [16]:
from simpletransformers.ner import NERModel,NERArgs

In [17]:
label = data["labels"].unique().tolist()
label

['O', 'BASE']

In [27]:
args = NERArgs()
args.num_train_epochs = 1000
args.learning_rate = 1e-4
args.overwrite_output_dir =True
args.train_batch_size = 32
args.eval_batch_size = 32

In [28]:
model = NERModel('bert', 'bert-base-cased',labels=label,args =args)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

In [ ]:
model.train_model(train_data,eval_data = test_data,acc=accuracy_score)

  0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1000 [00:00<?, ?it/s]

Running Epoch 0 of 1000:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


...

Running Epoch 13 of 1000:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 14 of 1000:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 15 of 1000:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 16 of 1000:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 17 of 1000:   0%|          | 0/2 [00:00<?, ?it/s]

In [21]:
result, model_outputs, preds_list = model.eval_model(test_data)

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: BASE seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


In [22]:
result

{'eval_loss': 0.5754972100257874,
 'precision': 1.0,
 'recall': 0.08108108108108109,
 'f1_score': 0.15}

In [23]:
"""
for x in test_data['sentence_id']:
  for i, s in zip(data['sentence_id'], data['text']):
    if i == x:
      prediction, model_output = model.predict([s])
      for x in prediction[0]:
        for key, val in x.items():
          if val == "BASE":
            print(key)
"""

'\nfor x in test_data[\'sentence_id\']:\n  for i, s in zip(data[\'sentence_id\'], data[\'text\']):\n    if i == x:\n      prediction, model_output = model.predict([s])\n      for x in prediction[0]:\n        for key, val in x.items():\n          if val == "BASE":\n            print(key)\n'

In [24]:
prediction, model_output = model.predict(["In recent years, disease and other disturbances have caused forests to die, emitting carbon dioxide instead as they rot. IMPLIED_BASE IMPLIED_BASE"])

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [25]:
test_data

,sentence_id,words,labels
360,28,kill,O
370,29,Smoke,BASE
264,18,IMPLIED_BASE,BASE
500,41,climate change,BASE
593,48,concentrations,O
...,...,...,...
633,50,have,O
683,56,As,O
246,17,caused,O
523,42,violence,BASE


In [26]:
prediction[0]
#for x in prediction[0]:
#  for key, val in x.items():
#    if val == "BASE":
#      print(key)

[{'In': 'O'},
 {'recent': 'O'},
 {'years,': 'O'},
 {'disease': 'O'},
 {'and': 'O'},
 {'other': 'O'},
 {'disturbances': 'O'},
 {'have': 'O'},
 {'caused': 'O'},
 {'forests': 'O'},
 {'to': 'O'},
 {'die,': 'O'},
 {'emitting': 'O'},
 {'carbon': 'O'},
 {'dioxide': 'O'},
 {'instead': 'O'},
 {'as': 'O'},
 {'they': 'O'},
 {'rot.': 'O'},
 {'IMPLIED_BASE': 'O'},
 {'IMPLIED_BASE': 'O'}]